In [92]:

import requests
import xml.etree.ElementTree as ET
import pandas as pd
df = pd.read_pickle("../data/arxiv_paper_ids.pickle")

ARXIV_LINK= "http://export.arxiv.org/api/query?search_query="
ns={'n':'http://www.w3.org/2005/Atom'}

def construct_arxiv_link(paper_title=None, author=None, abstract=None, start_idx = 0, max_results= 100):
    param_dict= {
        "ti": paper_title,
        "au":author,
        "abs": abstract,
        }
    str_query = ""
    query_list = []
    for k, v in param_dict.items():
        if v is not None:
            value_str = "\ ".join(v.split(" "))
            query_list.append(f"{k}:{value_str}")
    str_query = "+AND+".join(query_list)

        

    str_query += f"&sortBy=lastUpdatedDate&start={start_idx}&max_results={max_results}"
    return ARXIV_LINK + str_query

In [107]:
paper_title=  None
author= None
abstract= "image features"
link_arxiv = construct_arxiv_link(paper_title=paper_title, author=author,abstract=abstract, max_results=200)

response = requests.get(link_arxiv)
xmlstring = response.text
tree = ET.ElementTree(ET.fromstring(xmlstring))
tree_root = tree.getroot()
all_papers = tree_root.findall('n:entry',namespaces=ns)
papers_data = {}
for paper in all_papers:
    temp_tile = paper.find('n:title',namespaces=ns).text
    all_authors = list(paper.findall('n:author',namespaces=ns))
    paper_id = paper.find('n:id',namespaces=ns).text.replace("http://arxiv.org/","")
    paper_author_list = []
    for au in all_authors:
        paper_author_list.append(list(au)[0].text)
    paper_details = {"paper_arxiv_id":paper_id, "authors": paper_author_list, "paper_title": temp_tile}
    papers_data[paper_id] = paper_details


In [108]:
for k,v in papers_data.items():
    temp_df = pd.DataFrame.from_dict(papers_data[k])
    df = pd.concat([df, temp_df])
df = df.drop_duplicates()
df.to_pickle("../data/arxiv_paper_ids.pickle")

In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle("../data/arxiv_paper_ids.pickle")

In [14]:
df["summary"] = ""

In [15]:
unq_paper_id = df["paper_arxiv_id"].unique()

In [16]:
for unq_id in unq_paper_id:
    id_paper = unq_id.replace("abs/", "")
    query_arxiv_abstract = f"http://export.arxiv.org/api/query?id_list={id_paper}"
    out_id = requests.get(query_arxiv_abstract)
    tree = ET.ElementTree(ET.fromstring(out_id.text))
    tree_root = tree.getroot()
    paper = tree_root.findall('n:entry',namespaces=ns)[0]
    paper_summary = paper.find('n:summary',namespaces=ns).text
    summary = paper_summary.replace("\n","").lstrip().rstrip()
    df["summary"][df["paper_arxiv_id"] == unq_id] = summary
    print(unq_id)
    

abs/2212.00794v1
abs/2205.09113v2
abs/2203.16527v2
abs/2111.06377v3
abs/2111.11429v1
abs/2104.02057v4
abs/2104.14558v1
abs/2011.10566v1
abs/2007.06559v2
abs/2003.12056v2
abs/1912.00998v2
abs/2003.13678v1
abs/1911.05722v3
abs/2003.04297v1
abs/1912.08193v2
abs/1812.03982v3
abs/1903.12174v2
abs/1904.09664v2
abs/1812.05038v2
abs/1801.00868v3
abs/1901.02446v2
abs/1904.01569v2
abs/1812.03411v2
abs/1811.08883v1
abs/1806.05662v3
abs/1803.08494v3
abs/1805.00932v1
abs/1706.02677v2
abs/1711.07971v3
abs/1711.10370v2
abs/1704.07333v3
abs/1708.02002v2
abs/1703.06870v3
abs/1712.04440v1
abs/1708.02901v3
abs/1612.03144v2
abs/1611.05431v2
abs/1504.06066v2
abs/1607.07032v2
abs/1603.05027v3
abs/1605.06409v2
abs/1604.05144v1
abs/1603.08678v1
abs/1506.01497v3
abs/1512.04412v1
abs/1512.03385v1
abs/1505.06798v2
abs/1501.00092v3
abs/1503.01640v2
abs/1505.00996v1
abs/1406.4729v4
abs/1412.1283v4
abs/1502.01852v1
abs/1412.1710v1
abs/1411.4229v1
abs/2211.10831v1
abs/2206.07643v2
abs/2211.01340v2
abs/2210.16782v1
a

In [18]:
df.to_pickle("../data/arxiv_paper_ids.pickle")

In [1]:
from parsel import Selector
import os, json, requests, re
from fp.fp import FreeProxy
import time 
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle


In [3]:
df = pd.read_pickle("../data/arxiv_paper_ids.pickle")

In [4]:
authors = df.authors.unique()

In [5]:
sem_sch_author_cols = ["authorId", "name","aliases", "homepage","paperCount","citationCount","hIndex"]
sem_sch_paper_cols = ['authorId', 'name', 'paperId', 'url', 'title', 'abstract', 'year',
       'referenceCount', 'citationCount', 'influentialCitationCount',
       'isOpenAccess']
sem_scholdar_api_reqs = ["paperId","url","title","abstract", "year", "referenceCount",
                            "citationCount","influentialCitationCount",
                            "isOpenAccess"]

In [6]:
def create_author_link(author_name):
    authors_req = "name,aliases,homepage,paperCount,citationCount,hIndex"
    author_query = "+".join(author_name.lower().split(" "))
    url = f"https://api.semanticscholar.org/graph/v1/author/search?query={author_query}&fields={authors_req}"
    return url


In [ ]:
headers={"Content-Type":"text"}

In [7]:

# results_authors = requests.get(create_author_link("Kaiming He"))

In [21]:
df_sem_sch_authors = pd.DataFrame(columns=sem_sch_author_cols)

In [13]:
proxy = FreeProxy(rand=True, elite=True).get()

In [39]:
for i in tqdm(list(range(3021, len(authors))), desc="find authors"):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3538.102 Safari/537.36 Edge/18.19582',
    }

    if i%100 ==0:
        proxy = FreeProxy(rand=True, elite=True).get()

    results_authors = requests.get(create_author_link(authors[i]), headers=headers, proxies={"http": proxy})
    results = results_authors.json()

    _df_sem_sch_authors = pd.DataFrame(columns=sem_sch_author_cols)
    _df_sem_scholar_papers = pd.DataFrame(columns=sem_sch_paper_cols)


    for author_data in results["data"]:
        author_df = {k:v for k,v in author_data.items() if k in sem_sch_author_cols}
        
        if author_df["aliases"] is None:
            author_df["aliases"] = [author_df["name"]]
        author_df = pd.DataFrame.from_dict(author_df)
        _df_sem_sch_authors = pd.concat([author_df, _df_sem_sch_authors])
        paper_auth = pd.DataFrame(columns=sem_sch_paper_cols)

    df_sem_sch_authors = pd.concat([df_sem_sch_authors, _df_sem_sch_authors])
    df_sem_sch_authors.to_pickle("../data/sem_sch_authorsv2.pickle")

    time.sleep(9)

find authors:   3%|▎         | 149/5642 [43:50<26:56:31, 17.66s/it]


KeyError: 'data'

In [40]:
results

{'message': 'Internal Server Error'}

In [36]:
(5*60)/9

33.333333333333336

In [30]:
results

{'message': 'Too Many Requests'}

In [28]:
rr = requests.get(create_author_link(authors[i]), headers=headers, proxies={"http": proxy})

In [29]:
rr.json()

{'total': 1,
 'offset': 0,
 'data': [{'authorId': '3142556',
   'name': 'P. Sermanet',
   'aliases': ['P. Sermanet', 'Pierre Sermanet'],
   'homepage': None,
   'paperCount': 53,
   'citationCount': 42339,
   'hIndex': 27}]}

In [17]:
results_authors = requests.get(create_author_link(authors[i]))

In [19]:
create_author_link(authors[i])

'https://api.semanticscholar.org/graph/v1/author/search?query=yanghao+li&fields=name,aliases,homepage,paperCount,citationCount,hIndex'

In [18]:
results_authors.json()

{'message': 'Too Many Requests'}

In [1]:
paper_req

NameError: name 'paper_req' is not defined

In [60]:
paper_req = "fields=url,title,abstract,year,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy,citations,references&limit=800"
result_papers = requests.get(f"https://api.semanticscholar.org/graph/v1/author/2058350112/papers?{paper_req}")

In [ ]:
import requests
r=requests.get("http://www.example.com/", headers={"Content-Type":"text

In [11]:
df_sem_sch_authors = pd.read_pickle("../data/sem_sch_authors.pickle")
df_sem_scholar_papers = pd.DataFrame(columns=sem_sch_paper_cols)


In [13]:
failed_authors = []

In [14]:
authors_done = df_sem_sch_authors["author_searched"].unique()

In [3]:
arxiv_id = "1706.03762"

In [4]:
url = f"https://api.semanticscholar.org/graph/v1/paper/arXiv:{arxiv_id}"

In [10]:

import requests

In [7]:
import sys
sys.path.append("../")

In [8]:
from data.twitter_key import SemanticScholarCreds

In [12]:
res = requests.get(url, headers={"x-api-key":SemanticScholarCreds.API_KEY })

In [13]:
res.json()

{'paperId': '204e3073870fae3d05bcbc2f6a8e263d9b72e776',
 'title': 'Attention is All you Need'}

In [17]:
for i in progressbar(list(range(len(authors))), "find authors"):
    if authors[i] not in authors_done:
        try:
            results = sch.search_author(authors[i])
        except:
            failed_authors.append([i, authors[i]])
            continue
        _df_sem_sch_authors = pd.DataFrame(columns=sem_sch_author_cols)
        _df_sem_scholar_papers = pd.DataFrame(columns=sem_sch_paper_cols)


        for author_data in results._data:
            author_df = {k:v for k,v in author_data.items() if k in sem_sch_author_cols}
            
            if author_df["aliases"] is None:
                author_df["aliases"] = [author_df["name"]]
            author_df = pd.DataFrame.from_dict(author_df)
            _df_sem_sch_authors = pd.concat([author_df, _df_sem_sch_authors])
            paper_auth = pd.DataFrame(columns=sem_sch_paper_cols)
            for paper in author_data["papers"]:
                paper_deets = {k:v for k,v in paper.items() if k in sem_scholdar_api_reqs}
                more_authors = {k:v for k,v in paper.items() if k == "authors"}
                more_authors_df = pd.DataFrame.from_records(more_authors["authors"])
                paper_deets_df = pd.DataFrame([v for _,v in paper_deets.items()]).T
                paper_deets_df.columns = sem_scholdar_api_reqs
                paper_deets_df.index = range(1)
                more_authors_df["paperId"] = paper_deets_df["paperId"].values[0]
                more_authors_df["paperId"] = more_authors_df["paperId"].astype("object")
                paper_deets_df["paperId"] = paper_deets_df["paperId"].astype("object")
                more_papers_df =  more_authors_df.merge(paper_deets_df,on="paperId", how="left")
                paper_auth = pd.concat([paper_auth, more_papers_df])
            _df_sem_scholar_papers = pd.concat([_df_sem_scholar_papers, paper_auth])
        _df_sem_sch_authors["author_searched"] = authors[i]
        _df_sem_scholar_papers["author_searched"] = authors[i]

        df_sem_sch_authors = pd.concat([df_sem_sch_authors, _df_sem_sch_authors])
        df_sem_scholar_papers = pd.concat([df_sem_scholar_papers, _df_sem_scholar_papers])
        df_sem_sch_authors.to_pickle("../data/sem_sch_authors.pickle")
        df_sem_scholar_papers.to_pickle("../data/sem_sch_papers.pickle")
        time.sleep(2)

KeyboardInterrupt: 

In [12]:
for i,_ in progressbar(failed_authors, "find authors"):
    try:
        results = sch.search_author(authors[i])
    except:
        failed_authors.append([i, authors[i]])
        continue
    _df_sem_sch_authors = pd.DataFrame(columns=sem_sch_author_cols)
    _df_sem_scholar_papers = pd.DataFrame(columns=sem_sch_paper_cols)


    for author_data in results._data:
        author_df = {k:v for k,v in author_data.items() if k in sem_sch_author_cols}
        
        if author_df["aliases"] is None:
            author_df["aliases"] = [author_df["name"]]
        author_df = pd.DataFrame.from_dict(author_df)
        _df_sem_sch_authors = pd.concat([author_df, _df_sem_sch_authors])
        paper_auth = pd.DataFrame(columns=sem_sch_paper_cols)
        for paper in author_data["papers"]:
            paper_deets = {k:v for k,v in paper.items() if k in sem_scholdar_api_reqs}
            more_authors = {k:v for k,v in paper.items() if k == "authors"}
            more_authors_df = pd.DataFrame.from_records(more_authors["authors"])
            paper_deets_df = pd.DataFrame([v for _,v in paper_deets.items()]).T
            paper_deets_df.columns = sem_scholdar_api_reqs
            paper_deets_df.index = range(1)
            more_authors_df["paperId"] = paper_deets_df["paperId"].values[0]
            more_authors_df["paperId"] = more_authors_df["paperId"].astype("object")
            paper_deets_df["paperId"] = paper_deets_df["paperId"].astype("object")
            more_papers_df =  more_authors_df.merge(paper_deets_df,on="paperId", how="left")
            paper_auth = pd.concat([paper_auth, more_papers_df])
        _df_sem_scholar_papers = pd.concat([_df_sem_scholar_papers, paper_auth])
    _df_sem_sch_authors["author_searched"] = authors[i]
    _df_sem_scholar_papers["author_searched"] = authors[i]

    df_sem_sch_authors = pd.concat([df_sem_sch_authors, _df_sem_sch_authors])
    df_sem_scholar_papers = pd.concat([df_sem_scholar_papers, _df_sem_scholar_papers])
    df_sem_sch_authors.to_pickle("../data/sem_sch_authors.pickle")
    df_sem_scholar_papers.to_pickle("../data/sem_sch_papers.pickle")
    time.sleep(2)

KeyboardInterrupt: 

In [33]:
proxy = FreeProxy(rand=True, elite=True).get()
with requests.Session() as session:
   for proxy in proxies:
       send_request(session, proxy)

In [32]:
with requests.Session() as session:
   for proxy in proxies:
       send_request(session, proxy)

'http://119.8.236.97:3128'

In [6]:
authors_data = {}

In [2]:
from scholarly import ProxyGenerator
from scholarly import scholarly
# Set up a ProxyGenerator object to use free proxies
# This needs to be done only once per session
pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)

# Now search Google Scholar from behind a proxy
search_query = scholarly.search_pubs('Perception of physical stability and center of mass of 3D objects')
scholarly.pprint(next(search_query))

In [16]:
import numpy

def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]
        
levenshteinDistanceDP("kelm", "hello")

3.0